In [ ]:
from esper.prelude import *
import json

In [ ]:
videos = list(Video.objects.all()[:180000])

In [ ]:
def check_meta(v):
    try:
        meta = json.load(open('/app/data/ia-metadata/{}.json'.format(v.item_name())))
        runtime = meta['metadata']['runtime']
        [h, m, s] = [int(s) for s in runtime.split(':')]
        computed_dur = v.num_frames / v.fps
        expected_dur = h * 3600 + m * 60 + s
        return (v, computed_dur, expected_dur)
    except Exception:
        return None
durs = par_for(check_meta, videos, workers=16)

In [ ]:
i = 0
for (v, computed_dur, expected_dur) in durs:
    if abs(computed_dur - expected_dur) > 100:
        print('{}: {:.0f}, {:.0f}'.format(v.item_name(), computed_dur, expected_dur))

In [ ]:
Video.objects.get(path__contains='CNN_20111120_000000_CNN_Newsroom')